In [235]:
import json
import pandas as pd
import time
from collections import Counter, defaultdict

In [203]:
events_by_loc = json.load(open('music200.json', 'r'))

In [237]:
today = time.strftime("%Y-%m-%d")

In [247]:
e['_embedded']['venues']

[{'_links': {'self': {'href': '/discovery/v2/venues/ZFr9jZaedv?locale=en-us'}},
  'address': {'line1': '537 West 100 South'},
  'city': {'name': 'Salt Lake City'},
  'country': {'countryCode': 'US', 'name': 'United States Of America'},
  'id': 'ZFr9jZaedv',
  'locale': 'en-us',
  'location': {'latitude': '40.782001', 'longitude': '-111.7519'},
  'name': 'The Complex - UT',
  'postalCode': '84108',
  'state': {'name': 'Utah', 'stateCode': 'UT'},
  'test': False,
  'timezone': 'America/Denver',
  'type': 'venue',
  'upcomingEvents': {'_total': 35, 'tmr': 35}}]

In [259]:
rows = []
for _, info in events_by_loc.items():
    loc_name = info['name']
    
    # print(info)
    if 'coordinates' not in info:
        continue
        
    coord = info['coordinates']
    events = json.loads(info['events'])
    if events['page']['totalElements']:
        for e in events['_embedded']['events']:
            if 'classifications' in e:
                if ('attractions' in e['_embedded'] and len(e['_embedded']['attractions']) > 0
                    and len(e['_embedded']['venues']) > 0 and 'address' in e['_embedded']['venues'][0]):
                    if 'genre' in e['classifications'][0]:
                        topic = e['classifications'][0]['genre']['name']
                        if topic != 'Undefined' and topic != 'Alternative' and topic != 'Other':
                            date = e['dates']['start']['localDate']
                            venue = e['_embedded']['venues'][0]
                            line = venue['address'].get('line1') or venue['address'].get('line2')
                            if line is None:
                                continue
                            address = (line + ', ' + venue['city']['name'])
                            if date != today:
                                rows.append({
                                    'name': e['name'], 
                                    'url': e['url'], 
                                    'date': date,
                                    'country': info['country'],
                                    'city': loc_name,
                                    'address': address,
                                    'attractions': [a['name'] for a in e['_embedded']['attractions'] if 'name' in a],
                                    'coord': {'lon': float(coord['lon']), 'lat': float(coord['lat'])},
                                    'topic': topic
                                })


In [261]:
topics = Counter()
countries = Counter()
cities = Counter()
city2loc = {}

for r in rows:
    topics[r['topic']] += 1
    countries[r['country']] += 1
    cities[r['city']] += 1
    city2loc[r['city']] = r['coord']

In [262]:
rows_grouped = defaultdict(list)
for r in rows:
    rows_grouped[r['city']].append(r)

In [263]:
json.dump(city2loc, open('city2loc.json', 'w'))

In [264]:
json.dump(rows_grouped, open('events_grouped.json', 'w'))

In [265]:
json.dump(rows, open('rows.json', 'w'))